<a href="https://colab.research.google.com/github/VaeriaBautista/Simulaci-n-I/blob/main/generador_del_cuadrado_medio_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tabulate

def cuadrados_medios(semilla_inicial, cantidad_maxima):
    """
    Genera números pseudoaleatorios utilizando el método de cuadrados medios.

    Args:
    semilla_inicial (int): Número inicial
    cantidad_maxima (int): Número máximo de intentos de generación

    Returns:
    tuple: Lista de detalles de generación, ciclo de vida y motivo de detención
    """
    detalles_generacion = []
    semillas_usadas = {}  # Diccionario para rastrear las semillas y su iteración
    semilla = semilla_inicial
    ciclo_vida = []

    for iteracion in range(cantidad_maxima):
        # Elevar al cuadrado la semilla
        cuadrado = semilla ** 2

        # Convertir a cadena y rellenar con ceros a la izquierda
        cuadrado_str = str(cuadrado).zfill(8)

        # Extraer los 4 números centrales según sea impar o par el número de dígitos
        if len(cuadrado_str) % 2 == 1:  # Número de dígitos impar
            # Dejar el primer dígito y tomar los siguientes 4 como centrales
            semilla = int(cuadrado_str[1:5])
        else:  # Número de dígitos par
            # Tomar los 4 dígitos exactamente del centro
            inicio = (len(cuadrado_str) - 4) // 2
            semilla = int(cuadrado_str[inicio:inicio+4])

        # Calcular el valor normalizado (entre 0 y 1)
        normalizado = semilla / 10000

        # Condición 1: Detener si los 4 números centrales son cero
        if semilla == 0:
            ciclo_vida = identificar_ciclo_vida(detalles_generacion)
            return detalles_generacion, ciclo_vida, "Detenido: Semilla se convirtió en cero"

        # Condición 2: A partir de la segunda iteración, detener si la semilla se repite
        if iteracion > 0 and semilla in semillas_usadas:
            # Identificar el ciclo de vida
            ciclo_inicio = semillas_usadas[semilla]
            ciclo_vida = identificar_ciclo_vida(detalles_generacion, ciclo_inicio, iteracion)
            return detalles_generacion, ciclo_vida, f"Detenido: Semilla {semilla} repetida en iteraciones {ciclo_inicio+1} y {iteracion+1}"

        # Añadir la semilla al diccionario de semillas usadas con su iteración
        semillas_usadas[semilla] = iteracion

        # Preparar detalles de la iteración
        detalle = {
            'Iteración': iteracion + 1,
            'Semilla': semilla,
            'Semilla al Cuadrado': cuadrado,
            'Normalizado': normalizado
        }

        # Añadir a la lista de detalles
        detalles_generacion.append(detalle)

    ciclo_vida = identificar_ciclo_vida(detalles_generacion)
    return detalles_generacion, ciclo_vida, "Detenido: Alcanzado número máximo de iteraciones"

def identificar_ciclo_vida(detalles, ciclo_inicio=None, ciclo_fin=None):
    """
    Identifica y clasifica el ciclo de vida de la secuencia.

    Args:
    detalles (list): Lista de detalles de las iteraciones
    ciclo_inicio (int, opcional): Índice donde comienza el ciclo
    ciclo_fin (int, opcional): Índice donde termina el ciclo

    Returns:
    dict: Información del ciclo de vida
    """
    # Extraer las semillas
    semillas = [detalle['Semilla'] for detalle in detalles]

    # Si no hay ciclo identificado explícitamente, intentar encontrar uno
    if ciclo_inicio is None or ciclo_fin is None:
        for i, semilla in enumerate(semillas):
            if semilla in semillas[:i]:
                ciclo_inicio = semillas.index(semilla)
                ciclo_fin = i
                break

    # Calcular longitudes
    total_iteraciones = len(semillas)

    # Inicializar la información del ciclo de vida
    ciclo_info = {
        "Total de Iteraciones": total_iteraciones,
        "Longitud Total": total_iteraciones,
        "Semilla Inicial": detalles[0]['Semilla'] if detalles else None
    }

    # Si se encontró un ciclo
    if ciclo_inicio is not None and ciclo_fin is not None:
        longitud_ciclo = ciclo_fin - ciclo_inicio
        ciclo_info.update({
            "Tipo de Ciclo": "Ciclo identificado",
            "Inicio del Ciclo": ciclo_inicio + 1,
            "Fin del Ciclo": ciclo_fin + 1,
            "Longitud del Ciclo": longitud_ciclo,
            "Periodo Transitorio": ciclo_inicio,
            "Semillas en el Ciclo": semillas[ciclo_inicio:ciclo_fin]
        })
    else:
        ciclo_info["Tipo de Ciclo"] = "Sin ciclo identificado"

    return ciclo_info

# Ejemplo de uso
def main():
    # Semilla inicial de 4 dígitos
    semilla_inicial = 1938

    # Generar números aleatorios con límite de 20 iteraciones
    detalles, ciclo_vida, razon = cuadrados_medios(semilla_inicial, 200)

    # Generar tabla de detalles
    print("Tabla de Generación de Números Pseudoaleatorios:")
    print(tabulate.tabulate(
        detalles,
        headers="keys",
        tablefmt="grid"
    ))

    print("\n" + razon)

    # Mostrar información del ciclo de vida
    print("\nCiclo de Vida:")
    for clave, valor in ciclo_vida.items():
        print(f"{clave}: {valor}")

# Ejecutar el ejemplo
if __name__ == "__main__":
    main()

Tabla de Generación de Números Pseudoaleatorios:
+-------------+-----------+-----------------------+---------------+
|   Iteración |   Semilla |   Semilla al Cuadrado |   Normalizado |
+=============+===========+=======================+===============+
|           1 |      7558 |               3755844 |        0.7558 |
+-------------+-----------+-----------------------+---------------+
|           2 |      1233 |              57123364 |        0.1233 |
+-------------+-----------+-----------------------+---------------+
|           3 |      5202 |               1520289 |        0.5202 |
+-------------+-----------+-----------------------+---------------+
|           4 |       608 |              27060804 |        0.0608 |
+-------------+-----------+-----------------------+---------------+
|           5 |      3696 |                369664 |        0.3696 |
+-------------+-----------+-----------------------+---------------+
|           6 |      6604 |              13660416 |        0.6604 |